<a href="https://colab.research.google.com/github/doganozen/Retail_Content_Based_Recommendation/blob/main/retail_content_based_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import pandas as pd
pd.set_option('display.max_columns', 30)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df_products = pd.read_csv("products.csv")

df_products.head()

# Define the sample size
sample_size = 5000  # Data should be diminished because of processing of cosinus similarity. Otherwise kernel will restart

# Simple random sampling
sample_df = df_products.sample(sample_size, replace=False)

sample_df = sample_df.set_index("Product_ID").reset_index()

sample_df.head()


,Product_ID,name,main_category,sub_category,ratings,no_of_ratings,PRICE
0,9446,FREXCEZ USB 3.0 Type C Female to USB A Male Co...,Electronics,Electronic Gadgets,3.5,93,AUD 18.85
1,33562,Men's Skinny Fit Denim Mid Rise Jeans (Dark Bl...,men's clothing,Jeans,2.9,764,AUD 39.34
2,28776,Van Heusen Denim Labs Men's Slim Jeans (VXDNUP...,men's clothing,Jeans,3.4,859,AUD 56.58
3,67519,"Upix® AC Remote No. 36L, Compatible for LG AC ...",Electronics,Home Entertainment Systems,5.0,1,AUD 16.96
4,87769,CROSS Men's Leather Belt & Wallet Combo Gift Set,stores,Men's Fashion,4.1,73,AUD 66.02


In [74]:
def calculate_cosine_sim(dataframe_col):
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(dataframe_col)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim


In [75]:
def content_based_recommender(ID, cosine_sim, dataframe):
    # index'leri olusturma
    dataframe = dataframe[~dataframe["name"].isna()]
    indices = pd.Series(dataframe.index, index=dataframe['Product_ID'])
    # indices = indices[~indices.index.duplicated(keep='last')]

    # ID'nin index'ini yakalama
    product_index = indices[ID]

    # title'a gore benzerlik skorlarını hesapalama
    similarity_scores = pd.DataFrame(cosine_sim[product_index], columns=["score"])
    # kendisi haric ilk 10 ürünü getirme
    product_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
    return dataframe[['Product_ID','name','main_category','sub_category']].iloc[product_indices]


sample_df.loc[0:1]
# content_based_recommender(7676, cosine_sim, sample_df)



,Product_ID,name,main_category,sub_category,ratings,no_of_ratings,PRICE
0,9446,FREXCEZ USB 3.0 Type C Female to USB A Male Co...,Electronics,Electronic Gadgets,3.5,93,AUD 18.85
1,33562,Men's Skinny Fit Denim Mid Rise Jeans (Dark Bl...,men's clothing,Jeans,2.9,764,AUD 39.34


In [85]:
pd.set_option('display.max_colwidth', None)
cosine_sim_with_name = calculate_cosine_sim(sample_df['name'])
content_based_recommender(sample_df["Product_ID"].loc[0], cosine_sim_with_name, sample_df)

,Product_ID,name,main_category,sub_category
165,105901,"RS Micro HDMI Adapter, HDMI Female (Type-A) to Micro HDMI Male (Type-D) Gold Plated Connector Converter Adapter (Not Micr...",Electronics,Cameras
954,64085,"Tizum USB Type C to Type C 5A (100W) Braided Sync & Fast Charging Cable for Mobile, Laptop, Macbook & other Type-C devices...",Electronics,Home Entertainment Systems
2352,72933,"Fast Type-C USB Cable for vivo Y75 5G , vivo Y 75 5G Double Braided Nylon USB Type-C to Type-A 2.0 Male Cable | USB Cable ...",Electronics,Home Entertainment Systems
4227,68201,IGADG USB Keychain with Type C Connector (Black),Electronics,Home Entertainment Systems
1995,65868,USB Type C Cable for Samsung Galaxy Note20 Ultra 5G Original USB Type C to USB A Male Cable Best Tangle Free Heavy Duty Hi...,Electronics,Home Entertainment Systems
4937,64151,"tizum USB Type C to USB 3.0 OTG Cable/Adapter/Connector, Fast Charging/Data Transfer, Speed Upto 4.8 GBPS, Compatible with...",Electronics,Home Entertainment Systems
3119,72970,A2ZSHOP USB Type-C Cable Compatible for Motorola Moto G6 USB Cable Type-C Cable | C-Type Charger Cable | Type C Rapid Quic...,Electronics,Home Entertainment Systems
2043,71064,LipiWorld® USB 2.0 Active Extension Repeater Cable Male to Female Cord Adapter for All Extend USB Devices-Printer/Scanner...,Electronics,Home Entertainment Systems
1731,747,"Oneplus 10R Type c to 3.5mm Audio Jack, 1-Plus Audio Jack Adapter, car Audio Connector USB Type c, Headphone Converter Cab...",Electronics,Electronic Gadgets
2347,2196,"AmazonBasics USB Type-C to USB Type-C 2.0 Fast Charging Cable for Smartphone, Laptop - 6 Feet (1.8 Meters), White",Electronics,Electronic Gadgets


In [86]:
content_based_recommender(sample_df["Product_ID"].loc[1], cosine_sim_with_name, sample_df)

,Product_ID,name,main_category,sub_category
1,33562,"Men's Skinny Fit Denim Mid Rise Jeans (Dark Blue, 42)-PID42911",men's clothing,Jeans
2362,33703,"Men's Skinny Fit Denim Mid Rise Jeans (Black, 42)-PID42909",men's clothing,Jeans
2267,32543,"Men's Slim Fit Denim Mid Rise Stretchable Jeans (Black, 42)-PID42892",men's clothing,Jeans
3999,32936,"Men's Skinny Fit Denim Mid Rise Jeans (Blue, 50)-PID42910",men's clothing,Jeans
3758,33796,"Men's Skinny Fit Denim Mid Rise Jeans (Blue, 40)-PID42910",men's clothing,Jeans
767,33691,"Men's Slim Fit Denim Mid Rise Stretchable Jeans (Grey, 42)-PID42905",men's clothing,Jeans
2047,32783,"Men's Slim Fit Denim Mid Rise Jeans (Dark Blue, 34)-PID42908",men's clothing,Jeans
3387,33604,"Men's Slim Fit Denim Mid Rise Jeans (Dark Blue, 50)-PID42908",men's clothing,Jeans
2277,33533,"Men's Slim Fit Denim Mid Rise Jeans (Dark Blue, 50)-PID42908",men's clothing,Jeans
3879,32274,"Men's Skinny Fit Denim Mid Rise Jeans (Blue, 38)-PID42910",men's clothing,Jeans



###############################################

These 3 columns can be combined: name, main_category, sub_category.
So, we can recommend more accurate products because category columns give critical information about products
###############################################

In [87]:
sample_df['concated_info'] = sample_df['main_category'] + ' ' + sample_df['sub_category'] + ' ' + sample_df['name']

In [88]:
sample_df.head()

,Product_ID,name,main_category,sub_category,ratings,no_of_ratings,PRICE,concated_info
0,9446,FREXCEZ USB 3.0 Type C Female to USB A Male Connector/Converter/Adapter Compatible for All Type-C Devices (Multi-Color),Electronics,Electronic Gadgets,3.5,93,AUD 18.85,Electronics Electronic Gadgets FREXCEZ USB 3.0 Type C Female to USB A Male Connector/Converter/Adapter Compatible for All Type-C Devices (Multi-Color)
1,33562,"Men's Skinny Fit Denim Mid Rise Jeans (Dark Blue, 42)-PID42911",men's clothing,Jeans,2.9,764,AUD 39.34,"men's clothing Jeans Men's Skinny Fit Denim Mid Rise Jeans (Dark Blue, 42)-PID42911"
2,28776,Van Heusen Denim Labs Men's Slim Jeans (VXDNUPOFF46547_Navy_32),men's clothing,Jeans,3.4,859,AUD 56.58,men's clothing Jeans Van Heusen Denim Labs Men's Slim Jeans (VXDNUPOFF46547_Navy_32)
3,67519,"Upix® AC Remote No. 36L, Compatible for LG AC Remote Control (Exactly Same Remote Will Only Work)",Electronics,Home Entertainment Systems,5.0,1,AUD 16.96,"Electronics Home Entertainment Systems Upix® AC Remote No. 36L, Compatible for LG AC Remote Control (Exactly Same Remote Will Only Work)"
4,87769,CROSS Men's Leather Belt & Wallet Combo Gift Set,stores,Men's Fashion,4.1,73,AUD 66.02,stores Men's Fashion CROSS Men's Leather Belt & Wallet Combo Gift Set


In [89]:
cosine_sim_with_concated_info = calculate_cosine_sim(sample_df['concated_info'])
content_based_recommender(sample_df["Product_ID"].loc[0], cosine_sim_with_concated_info, sample_df)

,Product_ID,name,main_category,sub_category
165,105901,"RS Micro HDMI Adapter, HDMI Female (Type-A) to Micro HDMI Male (Type-D) Gold Plated Connector Converter Adapter (Not Micr...",Electronics,Cameras
2347,2196,"AmazonBasics USB Type-C to USB Type-C 2.0 Fast Charging Cable for Smartphone, Laptop - 6 Feet (1.8 Meters), White",Electronics,Electronic Gadgets
1731,747,"Oneplus 10R Type c to 3.5mm Audio Jack, 1-Plus Audio Jack Adapter, car Audio Connector USB Type c, Headphone Converter Cab...",Electronics,Electronic Gadgets
954,64085,"Tizum USB Type C to Type C 5A (100W) Braided Sync & Fast Charging Cable for Mobile, Laptop, Macbook & other Type-C devices...",Electronics,Home Entertainment Systems
2352,72933,"Fast Type-C USB Cable for vivo Y75 5G , vivo Y 75 5G Double Braided Nylon USB Type-C to Type-A 2.0 Male Cable | USB Cable ...",Electronics,Home Entertainment Systems
1995,65868,USB Type C Cable for Samsung Galaxy Note20 Ultra 5G Original USB Type C to USB A Male Cable Best Tangle Free Heavy Duty Hi...,Electronics,Home Entertainment Systems
3119,72970,A2ZSHOP USB Type-C Cable Compatible for Motorola Moto G6 USB Cable Type-C Cable | C-Type Charger Cable | Type C Rapid Quic...,Electronics,Home Entertainment Systems
4937,64151,"tizum USB Type C to USB 3.0 OTG Cable/Adapter/Connector, Fast Charging/Data Transfer, Speed Upto 4.8 GBPS, Compatible with...",Electronics,Home Entertainment Systems
2043,71064,LipiWorld® USB 2.0 Active Extension Repeater Cable Male to Female Cord Adapter for All Extend USB Devices-Printer/Scanner...,Electronics,Home Entertainment Systems
2606,2711,Choseal USB Type C to 3.5 mm Audio Jack Connector with Hi-Res DAC Headphone Converter Adapter for Samsung Galaxy S22/S21/S...,Electronics,Electronic Gadgets


In [90]:
content_based_recommender(sample_df["Product_ID"].loc[1], cosine_sim_with_concated_info, sample_df)

,Product_ID,name,main_category,sub_category
1,33562,"Men's Skinny Fit Denim Mid Rise Jeans (Dark Blue, 42)-PID42911",men's clothing,Jeans
2362,33703,"Men's Skinny Fit Denim Mid Rise Jeans (Black, 42)-PID42909",men's clothing,Jeans
2267,32543,"Men's Slim Fit Denim Mid Rise Stretchable Jeans (Black, 42)-PID42892",men's clothing,Jeans
3999,32936,"Men's Skinny Fit Denim Mid Rise Jeans (Blue, 50)-PID42910",men's clothing,Jeans
3758,33796,"Men's Skinny Fit Denim Mid Rise Jeans (Blue, 40)-PID42910",men's clothing,Jeans
3387,33604,"Men's Slim Fit Denim Mid Rise Jeans (Dark Blue, 50)-PID42908",men's clothing,Jeans
2277,33533,"Men's Slim Fit Denim Mid Rise Jeans (Dark Blue, 50)-PID42908",men's clothing,Jeans
2047,32783,"Men's Slim Fit Denim Mid Rise Jeans (Dark Blue, 34)-PID42908",men's clothing,Jeans
767,33691,"Men's Slim Fit Denim Mid Rise Stretchable Jeans (Grey, 42)-PID42905",men's clothing,Jeans
3879,32274,"Men's Skinny Fit Denim Mid Rise Jeans (Blue, 38)-PID42910",men's clothing,Jeans
